In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import tkinter as tk
from tkinter import filedialog, ttk
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import mplcursors  # Importing the mplcursors library

class EVM_app(tk.Tk):

    def __init__(self):
        super().__init__()
    
        #Initialize variables
        self.data = None  # Initialize data variable
        self.years = []  # Store unique years
        self.title("EVM Impacts Assessment")

        # Set the window to full screen
        screen_width = self.winfo_screenwidth()
        screen_height = self.winfo_screenheight()
        self.geometry(f"{screen_width}x{screen_height}")

        # Configure the grid
        self.grid_columnconfigure(0, weight=1)
        self.grid_columnconfigure(1, weight=1)
        self.grid_columnconfigure(2, weight=1)
        self.grid_columnconfigure(3, weight=1)
        self.grid_columnconfigure(4, weight=1)
        self.grid_columnconfigure(5, weight=1)
        self.grid_columnconfigure(6, weight=1)
        self.grid_columnconfigure(7, weight=1)
        self.grid_columnconfigure(8, weight=1)
        self.grid_columnconfigure(9, weight=1)

        self.grid_rowconfigure(15, weight=1)

        # Initialize chart frames (left and right)
        #self.left_frame = tk.Frame(self)
        #self.left_frame.grid(row=8, column=0, columnspan=3, sticky="nsew")
        
        #self.right_frame = tk.Frame(self)
        #self.right_frame.grid(row=8, column=4, columnspan=3, sticky="nsew")

        #Activate upload data button
        self.upload_data_button()
    
    def upload_data_button(self):

        # Add upload buttons
        upload_button = tk.Button(self, text="Upload Dataset", command=self.upload_dataset)
        upload_button.grid(row=1, column=1,  pady=(10, 10), sticky="ew")
    
    def upload_dataset(self):
        file_path = filedialog.askopenfilename(title="Select CSV file", filetypes=[("CSV files", "*.csv")])
        if file_path:
            self.data = pd.read_csv(file_path)

            # Ensure the 'Date' column is parsed as datetime
            self.data['Date'] = pd.to_datetime(self.data['Date'])

            # Extract unique years
            self.years = self.data['Date'].dt.year.unique().tolist()

            self.upload_attrib_data_button()

    def upload_attrib_data_button(self):
        upload_attrib_button = tk.Button(self, text="Upload Attributes", command=self.upload_attrib_dataset)
        upload_attrib_button.grid(row=2, column=1,  pady=(10, 10), sticky="ew")
    
    def upload_attrib_dataset(self):
        file_path = filedialog.askopenfilename(title="Select CSV file", filetypes=[("CSV files", "*.csv")])
        if file_path:
            self.attribute_data = pd.read_csv(file_path)

            # Run code to add dropdowns and populate them
            self.create_filters()
            self.update_dropdowns()

    def create_filters(self):

        # Create input fields for filtering
        self.create_filter_field("Filter Item Number:", row=1, column=2)
           
    def create_filter_field(self, label_text, row, column):
        """Helper function to create a label and dropdown field."""
        label = tk.Label(self, text=label_text)
        label.grid(row=row, column=column, padx=0, pady=5, sticky="e")

        combo = ttk.Combobox(self, state="readonly")
        combo.grid(row=row, column=column + 1, padx=0, pady=5, sticky="w")

        # Bind the combobox selection event
        combo.bind("<<ComboboxSelected>>", self.on_select)
        
        # Dynamically create instance variables to hold the input fields
        setattr(self, f"combo_{label_text.replace(' ', '_').replace(':', '')}", combo)

    def on_select(self, event):
        self.create_filtered_data()
       
    def update_dropdowns(self):
        """Update dropdowns with unique values from the dataset."""
        item_number_combo = getattr(self, "combo_Filter_Item_Number")

        # Populate item number dropdown
        item_numbers = self.data['Item Number'].unique().tolist()
        item_number_combo['values'] = item_numbers
        item_number_combo.set('')  # Set default selection to empty

    def create_filtered_data(self):
        # Get filter values
        filter_item_number = self.combo_Filter_Item_Number.get()

        filtered_data = self.data
        if filter_item_number:
            filtered_data = filtered_data[filtered_data['Item Number'].astype(str) == filter_item_number]
      
        # Ensure there's data to plot
        if filtered_data.empty:
            tk.messagebox.showwarning("Warning", "No data matches the filters.")
            print("Filtered Data is Empty")  # Debugging output
            return
        self.filtered_data = filtered_data

        self.verify_data_for_sliders()
        self.plot_charts()

    def verify_data_for_sliders(self):


        # Get selected item number
        filter_item_number = self.combo_Filter_Item_Number.get()

        # Check if the selected item exists in the attributes dataset
        if filter_item_number and not self.attribute_data.empty:
            selected_attributes = self.attribute_data[self.attribute_data['Item Number'].astype(str) == filter_item_number]
            if not selected_attributes.empty:
                # Extract relevant attributes for the sliders
                lead_time = selected_attributes['Lead Time'].values[0]
                yield_value = selected_attributes['Yield'].values[0]
                cost = selected_attributes['Cost'].values[0]
                hours = selected_attributes['Hours'].values[0]

                if hasattr(self, 'slider_frame'):
                    self.clear_sliders()


                # Create sliders with default values from the attributes dataset
                self.create_sliders(lead_time, yield_value, cost, hours)
            else:
                tk.messagebox.showwarning("Warning", "Selected item not found in the attributes file.")
        else:
            tk.messagebox.showwarning("Warning", "Attributes data not loaded.")
    
    def clear_sliders(self):
        """Clear the existing slider widgets before displaying new ones."""
        for widget in self.slider_frame.winfo_children():
            widget.destroy()

#--------------------------------------------------------------------------------
#-----------------Help Needed on This section please ----------------------------
#--------------------------------------------------------------------------------

    def create_sliders(self,lead_time, yield_value, cost, hours):
        
        """This code is executed each time a user selects a new item from the drop down menu
        you can uncomment the line below to either run it with the frame or without. If I add a frame everything stops making sense
        on how things are being located on the UI, but it does remove them before creating new ones"""
        #self.add_sliders_to_UI_within_frame(lead_time, yield_value, cost, hours)

        """This works exactly like I want, but it will continue stacking the slider bar elements on top of eachother and I can't figure out 
        how to delete them before recreating them for a different item number"""
        self.add_sliders_to_UI_without_frame(lead_time, yield_value, cost, hours)

#----------------- This is me trying to get the frames to work --------------------       
    def add_sliders_to_UI_within_frame(self, lead_time, yield_value, cost, hours):
        # Create a frame to hold sliders, if it doesn't exist
        if not hasattr(self, 'slider_frame'):
            self.slider_frame = tk.Frame(self, bd=3, relief="solid") #<------- added a border so I can see where this is located, this seems to work fine
            self.slider_frame.grid(row=1, column=4, rowspan=4, columnspan=4, sticky="nsew")

        """My desperate attempts here to figure something out, I feel like I have tried everything"""
        self.grid_columnconfigure(4, weight=1)
        self.grid_columnconfigure(5, weight=1)
        self.grid_columnconfigure(6, weight=1)
        self.grid_columnconfigure(7, weight=1)  
        self.grid_columnconfigure(8, weight=1)  
        self.grid_columnconfigure(9, weight=1)  
        self.grid_columnconfigure(10, weight=1)  

            # Configure the internal grid in the slider_frame to allow sliders and labels to expand
        self.slider_frame.grid_columnconfigure(0, weight=1)
        self.slider_frame.grid_columnconfigure(1, weight=1)
        self.slider_frame.grid_columnconfigure(2, weight=1)
        self.slider_frame.grid_columnconfigure(3, weight=2)
        self.slider_frame.grid_columnconfigure(4, weight=2)
        self.slider_frame.grid_columnconfigure(5, weight=2)
        self.slider_frame.grid_columnconfigure(6, weight=1)
        self.slider_frame.grid_rowconfigure(1, weight=1)
        self.slider_frame.grid_rowconfigure(2, weight=1)
        self.slider_frame.grid_rowconfigure(3, weight=1)
        self.slider_frame.grid_rowconfigure(4, weight=1)

        # Slider bars
        self.create_slider_within_frame("Leadtime: ", " days", lead_time, row=1, column=3, min=0, max=lead_time*10, default=lead_time)
        self.create_slider_within_frame("Yield: ", "%", yield_value, row=2, column=3, min=0, max=100, default=yield_value)
        self.create_slider_within_frame("Cost: $","", cost, row=3, column=3, min=0, max=cost*10, default=cost)
        self.create_slider_within_frame("Hours: "," hours", hours, row=4, column=3, min=0, max=hours*10, default=hours)

    def create_slider_within_frame(self, title, suffix, starting_value, row, column, min, max, default):

        # Create slider from 0 to 100 (representing percentages)
        slider = tk.Scale(self.slider_frame, from_=min, to=max, orient=tk.HORIZONTAL, showvalue=0)
        slider.grid(row=row, column=column+1, columnspan=3,padx=0, pady=10, sticky="ew")

        # Set default value to 0%
        slider.set(default)

        # Static starting value label
        starting_value_label = tk.Label(self.slider_frame, text=f"Starting {title}{starting_value}{suffix}")
        starting_value_label.grid(row=row, column=column, padx=0, pady=10, sticky="se")

        # Label to display percentage change result
        result_label = tk.Label(self.slider_frame, text=f"Modified {title} {slider.get()}{suffix}")
        result_label.grid(row=row, column=column+4, padx=0, pady=10, sticky="sw")

        # Update result label when slider is moved
        slider.bind("<Motion>", lambda event, sv=starting_value, sl=slider, rl=result_label: 
                    self.update_result_label(event, title, suffix, sl, rl, sv))
    

#----------------- This is what I had that was working before adding the frame --------------------       
    def add_sliders_to_UI_without_frame(self, lead_time, yield_value, cost, hours):



        """Help: This works exactly like I expect, it starts in column 4 and the slider bar has a column span of 3 and is reflected in the UI
        but I don't know how best to clear out the contents before recreating these fields"""
        # Slider bars
        self.create_slider_without_frame("Leadtime: ", " days", lead_time, row=1, column=4, min=0, max=lead_time*10, default=lead_time)
        self.create_slider_without_frame("Yield: ", "%", yield_value, row=2, column=4, min=0, max=100, default=yield_value)
        self.create_slider_without_frame("Cost: $","", cost, row=3, column=4, min=0, max=cost*10, default=cost)
        self.create_slider_without_frame("Hours: "," hours", hours, row=4, column=4, min=0, max=hours*10, default=hours)

    def create_slider_without_frame(self, title, suffix, starting_value, row, column, min, max, default):

        # Create slider from 0 to 100 (representing percentages)
        slider = tk.Scale(self, from_=min, to=max, orient=tk.HORIZONTAL, showvalue=0)
        slider.grid(row=row, column=column+1, columnspan=3,padx=0, pady=10, sticky="ew")

        # Set default value to 0%
        slider.set(default)

        # Static starting value label
        starting_value_label = tk.Label(self, text=f"Starting {title}{starting_value}{suffix}")
        starting_value_label.grid(row=row, column=column, padx=0, pady=10, sticky="se")

        # Label to display percentage change result
        result_label = tk.Label(self, text=f"Modified {title} {slider.get()}{suffix}")
        result_label.grid(row=row, column=column+4, padx=0, pady=10, sticky="sw")

        # Update result label when slider is moved
        slider.bind("<Motion>", lambda event, sv=starting_value, sl=slider, rl=result_label: 
                    self.update_result_label(event, title, suffix, sl, rl, sv))
    
    def update_result_label(self, event, title, suffix, slider, result_label, starting_value):
        """Update the result label based on the slider value and starting value."""
        result = self.calculate_percent_change(slider.get(), starting_value)
        result_label.config(text=f"Modified {title} {slider.get()}{suffix}")
    
#--------------------------------------------------------------------------------
#-----------------Help Needed on above section please ---------------------------
#--------------------------------------------------------------------------------

    def plot_charts(self):
        if self.data is None:
            tk.messagebox.showerror("Error", "No dataset uploaded.")
            return
        
        # Clear existing charts
        self.clear_frame(self.left_frame)
        self.clear_frame(self.right_frame)

        # Create a single plot for both cumulative line and bubble charts
        self.plot_cumulative_line_chart(self.filtered_data)
        self.plot_bubble_chart(self.filtered_data)

    def plot_cumulative_line_chart(self, filtered_data):
        fig, ax = plt.subplots()

        # Group by 'Column1' and plot cumulative costs
        for label, group in filtered_data.groupby('Column1'):
            cumulative_values = np.cumsum(group['Cost'].values)
            ax.plot(group['Date'].values, cumulative_values, marker='o', linestyle='-', label=label)

        # Format the y-axis as currency
        ax.yaxis.set_major_formatter(ticker.FuncFormatter(self.currency_format))

        # Adding labels, title, and legend
        plt.title('Cumulative Cost Over Time')
        plt.xlabel('Date')
        plt.ylabel('Cumulative Cost')
        plt.legend()

        # Embed the plot in Tkinter (left frame)
        canvas = FigureCanvasTkAgg(fig, master=self.left_frame)
        canvas.draw()
        canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)

        # Enable hover annotations for line chart
        mplcursors.cursor(hover=True)   
    
    def plot_line_chart_with_percent_delta(x_value_1, x_value_2, y_values_1, y_values_2, 
                                        data_label_1, data_label_2, x_label, y_label, chart_title):

        # Calculate percent delta
        percent_delta = [(y2 - y1) / y1 * 100 if y1 != 0 else None for y1, y2 in zip(y_values_1, y_values_2)]

        # Calculate BAC
        BAC = max(y_values_1)

        #Calculate EAC
        EAC = max(y_values_2)

        # Create the figure
        fig = go.Figure()

        # Add the first line (hoverinfo='skip' ensures it doesn't show on hover)
        fig.add_trace(go.Scatter(x=x_value_1, 
                                y=y_values_1, 
                                mode='lines', 
                                name=data_label_1, 
                                line=dict(color='blue'),
                                showlegend=False,
                                hoverinfo='skip'))  # Skip hover for this line

        # Add the second line (hoverinfo='skip' ensures it doesn't show on hover)
        fig.add_trace(go.Scatter(x=x_value_2, 
                                y=y_values_2, 
                                mode='lines', 
                                name=data_label_2, 
                                line=dict(color='green', 
                                        dash='dash'),
                                showlegend=False,
                                hoverinfo='skip'))  # Skip hover for this line

        #Add BAC as an annotation outside of the plot area
        fig.add_annotation(
            x=1.05,  # Position outside the plot area (in paper coordinates)        
            y=y_values_1[-1],
            text=f'BAC ${BAC:,.2f}',
            showarrow=False,
            yref = 'y',
            xref='paper',  # Reference the figure's width, not the data coordinates        
            xanchor="left",  # Align text to the left of the annotation point
            yanchor="middle"
        )

        #Add EAC as an annotation outside of the plot area
        fig.add_annotation(
            x=1.05,
            y=y_values_2[-1],
            text=f'EAC ${EAC:,.2f}',
            showarrow=False,
            yref = 'y',
            xref='paper',  # Reference the figure's width, not the data coordinates
            xanchor="left",  # Align text to the left of the annotation point
            yanchor="middle"
        )

    # Add a third trace for hover text with the percent delta only
        fig.add_trace(go.Scatter(
            x=x_value_1, 
            y=y_values_1, 
            mode='lines',
            line=dict(color='rgba(0,0,0,0)'), # Set the line color to transparent
            customdata=percent_delta,
            hovertemplate='Date: %{x}<br>Percent Delta: %{customdata:.2f}%<extra></extra>',
            showlegend=False))  # No legend entry for this trace

        # Customize the layout
        fig.update_layout(
            title=dict(
                text=chart_title,
                x=0.5,
                xanchor='center',
                yanchor='top'
            ),

            hovermode='x unified',
            yaxis_tickprefix='$',
            yaxis_tickformat=',.0f',
            showlegend=False,
            plot_bgcolor='white',  # Set the plot background to white
            paper_bgcolor='white',  # Set the overall chart background to white
            xaxis=dict(showgrid=True, 
                    gridcolor='lightgray'),  # Set grid lines for better visibility
            yaxis=dict(showgrid=True, 
                    gridcolor='lightgray'),
            margin=dict(l=80, 
                        r=150, 
                        t=50, 
                        b=50)  # Adjust 'r' for right margin size (150px in this example)
        )

        # Show the plot
        fig.show()
    
    def plot_bubble_chart(self, filtered_data):
        fig, ax = plt.subplots()

        # Plot bubble sizes for each entry in Column1
        for label, group in filtered_data.groupby('Column1'):
            total_cost = np.sum(group['Cost'].values)
            sizes = (group['Cost'].values / total_cost) * 1000  # Adjust size scale for bubbles

            # Use a constant value for y-position
            y_position = np.full(len(sizes), label)  # Create a constant array for y-position
            scatter = ax.scatter(group['Date'].values, y_position, s=sizes, alpha=0.5, label=label)

        # Set Y-axis to show unique values from Column1
        ax.set_yticks(filtered_data['Column1'].unique())
        ax.set_yticklabels(filtered_data['Column1'].unique())
        ax.set_ylim(-1, len(filtered_data['Column1'].unique()))  # Add space above and below the data

        # Add labels and title
        ax.set_title('Bubble Chart (Cost Distribution)')
        ax.set_xlabel('Date')
        ax.set_ylabel('Costs')

        # Embed the plot in Tkinter (right frame)
        canvas = FigureCanvasTkAgg(fig, master=self.right_frame)
        canvas.draw()
        canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)

        # Enable hover annotations for bubble chart
        mplcursors.cursor(scatter, hover=True)

    def clear_frame(self, frame):
        """Clear all widgets from a frame."""
        for widget in frame.winfo_children():
            widget.destroy()

    def currency_format(self, x, _):
        return '${:,.0f}'.format(x)


# Run the app
app = EVM_app()
app.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\CassieLynn\AppData\Local\Programs\Python\Python312\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\CassieLynn\AppData\Local\Temp\ipykernel_5924\4210944555.py", line 101, in on_select
    self.create_filtered_data()
  File "C:\Users\CassieLynn\AppData\Local\Temp\ipykernel_5924\4210944555.py", line 128, in create_filtered_data
    self.plot_charts()
  File "C:\Users\CassieLynn\AppData\Local\Temp\ipykernel_5924\4210944555.py", line 279, in plot_charts
    self.clear_frame(self.left_frame)
                     ^^^^^^^^^^^^^^^
  File "c:\Users\CassieLynn\AppData\Local\Programs\Python\Python312\Lib\tkinter\__init__.py", line 2411, in __getattr__
    return getattr(self.tk, attr)
           ^^^^^^^^^^^^^^^^^^^^^^
AttributeError: '_tkinter.tkapp' object has no attribute 'left_frame'


: 